In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd "/content/drive/MyDrive/Colab Notebooks"

import numpy as np
import os
import matplotlib.pyplot as plt 
import tensorflow as tf
from keras.utils import to_categorical
import csv
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,MaxPool2D,BatchNormalization
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.callbacks import TensorBoard

In [4]:
# Load dữ liệu
X_train = np.load('data/cv-cnn-ute-122/x_train.npy')
X_test = np.load('data/cv-cnn-ute-122/x_test.npy')
y_train = np.load('data/cv-cnn-ute-122/y_train.npy')

# 3. Reshape lại dữ liệu cho đúng kích thước mà keras yêu cầu
X_tr = X_train.reshape(50000, 32, 32,3 )
X_t = X_test.reshape(10000, 32, 32,3)

# X_tr = X_train.astype('float32')
# X_t = X_test.astype('float32')

X_tr = X_tr / 255.0
X_t = X_t / 255.0
y_train = to_categorical(y_train,10)

# định nghĩa hàm loss
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# định nghĩa thuật toán tối ưu
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
# optimizer = tf.keras.optimizers.RMSprop()
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# xây dựng mô hình
def create_model():
    model = Sequential()

# Convolutional Layer
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    # Pooling layer
    model.add(MaxPool2D(pool_size=(2, 2)))
    # Dropout layers
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
# model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))

    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

# def create_model():
#     model = Sequential()
#     model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
#     model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Dropout(0.2))
#     model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
#     model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Dropout(0.2))
#     model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
#     model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Dropout(0.2))
#     model.add(Flatten())
#     model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
#     model.add(Dropout(0.2))
#     model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
#     model.add(Dropout(0.2))
#     model.add(Dense(10, activation='softmax'))
#     # compile model
#     model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
#     return model

model = create_model()
model.summary()

# tensorboard = TensorBoard(
# 	  log_dir=os.path.join(os.getcwd(), "logs"),
# 	  histogram_freq=1,
# 	  write_images=True
# 	)

weights_filepath = './obj/'
callback = tf.keras.callbacks.ModelCheckpoint( filepath =
weights_filepath, monitor='val_loss', verbose=1,
save_best_only=False, save_weights_only=False)
# callback = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.01,
#     patience=10,
#     verbose=1,
#     min_delta=0.0001,
#     cooldown=0,
#     min_lr=0)
# bắt đầu training
his = model.fit(X_tr, y_train, epochs = 100, batch_size = 32,
validation_split=0.2, callbacks=callback)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                        

INFO:tensorflow:Assets written to: ./obj\assets


INFO:tensorflow:Assets written to: ./obj\assets


1250/1250 [==============================] - 133s 103ms/step - loss: 0.2457 - accuracy: 0.4537 - precision: 0.5855 - recall: 0.3249 - val_loss: 0.1836 - val_accuracy: 0.5955 - val_precision: 0.6795 - val_recall: 0.5246
Epoch 2/100
1250/1250 [==============================] - ETA: 0s - loss: 0.1724 - accuracy: 0.6370 - precision: 0.7274 - recall: 0.5562
Epoch 2: saving model to ./obj\


INFO:tensorflow:Assets written to: ./obj\assets


...

INFO:tensorflow:Assets written to: ./obj\assets


1250/1250 [==============================] - 88s 70ms/step - loss: 0.0179 - accuracy: 0.9719 - precision: 0.9756 - recall: 0.9686 - val_loss: 0.1059 - val_accuracy: 0.8621 - val_precision: 0.8765 - val_recall: 0.8544
Epoch 100/100
1250/1250 [==============================] - ETA: 0s - loss: 0.0184 - accuracy: 0.9718 - precision: 0.9757 - recall: 0.9689
Epoch 100: saving model to ./obj\


INFO:tensorflow:Assets written to: ./obj\assets


INFO:tensorflow:Assets written to: ./obj\assets


1250/1250 [==============================] - 91s 73ms/step - loss: 0.0184 - accuracy: 0.9718 - precision: 0.9757 - recall: 0.9689 - val_loss: 0.1059 - val_accuracy: 0.8613 - val_precision: 0.8749 - val_recall: 0.8531


In [31]:
# Load file mô hình đã huấn luyện
model = create_model()
model = tf.keras.models.load_model('./obj/')

prediction = model.predict(X_t)

out_np = np.argmax(prediction, axis=1)

f = open('output.csv', 'w', newline='')

headers = ['index', 'label']

with f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for i in range(out_np.shape[0]):
        writer.writerow([i,out_np[i]])

313/313 [==============================] - 1s 3ms/step
